In [1]:
import numpy as np
import os
from pImpactR import MLI as mli
from pImpactR import opt
from pImpactR.util import Me
from copy import deepcopy as copy
import time
import pickle

# Original Settings

In [2]:
elems0,latticeList,labor = mli.readInputfile('mli.in.origin')

setarclen is not recognized. skipping...
autoconcat is not recognized. skipping...
autoapply is not recognized. skipping...
reftraj is not recognized. skipping...
fit is not recognized. skipping...
bip is not recognized. skipping...
tip is not recognized. skipping...


In [3]:
energy=elems0[0].energy*1.0e9
gam0 = energy/Me
bet0 = np.sqrt(1.0-1.0/gam0**2)

In [4]:
elemList = []
for item in elems0:
    if not item.name in ['tasm','aim','vary','clear','anaprint']:
        elemList.append(item)

In [5]:
labor = ['iotaline','mapout','fin']

### Map

In [6]:
mli.writeInputfile(elemList,latticeList,labor)
mli.run()
M,G=mli.readTransferMap()

In [7]:
G.head()

,exponents,GP
28,f( 30 00 00 ),7.765582e-12
29,f( 21 00 00 ),-5.701697e-12
33,f( 20 00 01 ),7.272738e-01
34,f( 12 00 00 ),-2.987872e-12
38,f( 11 00 01 ),5.580342e-12


### Quadrupole settings

### Sextupoles

In [8]:
indexSext = []
SextStrength = []
for i,item in enumerate(elemList):
    if 'sextupole' == item.elem:
        indexSext.append(i)
        SextStrength.append(item.g2)
nSext = len(indexSext)

# Optimize

In [9]:
NL_nu = 0.3
NL_L  = 1.8
NL_c  = 0.01
NL_t  = 0.4
alfx = np.tan(np.pi*NL_nu)
betx = NL_L/np.sin(2.0*np.pi*NL_nu)
k = 2*alfx/betx

In [10]:
G2_ref = G.loc[28:76].copy()
G2_ref['GP'] = 0
G2_ref.loc[33,'GP'] = 0.5*k/bet0
G2_ref.loc[67,'GP'] = 0.5*k/bet0

G3_ref = G.loc[84:200].copy()
G3_ref['GP'] = 0
G3_ref.loc[104,'GP'] = k/(2.0*bet0*gam0)**2
G3_ref.loc[184,'GP'] = k/(2.0*bet0*gam0)**2

G4_ref = G.loc[210:450].copy()
G4_ref['GP'] = 0

G5_ref = G.loc[462:910].copy()
G5_ref['GP'] = 0

In [11]:
#%%
def objFunc(arg):     
    for i,j in enumerate(indexSext):
        elemList[j]['g2']=arg[i]
        
    mli.writeInputfile(elemList,latticeList,labor)
    mli.run()
    M,G = mli.readTransferMap()
    
    obj = 1.0e30* np.sum((G.loc[28 :76 ,'GP'].values-G2_ref['GP'].values)**2) \
         +1.0e22* np.sum((G.loc[84 :200,'GP'].values-G3_ref['GP'].values)**2) \
         +1.0e12* np.sum((G.loc[210:450,'GP'].values-G4_ref['GP'].values)**2) \
         +        np.sum((G.loc[462:910,'GP'].values-G5_ref['GP'].values)**2)
    return obj

In [12]:
scale = np.array([400.0]*nSext)
mid   = np.zeros(nSext)

In [13]:
import pickle
with open('result.sext','rb') as fp:
    result=pickle.load(fp)

In [14]:
result

     fun: 2.0107957494427288e+31
     jac: array([-4.15776821e+28, -1.89380868e+28, -1.54926079e+28,  1.75804767e+28,
       -4.96118787e+28, -5.87978708e+28, -2.49810168e+28,  1.57186886e+28,
       -1.24076422e+28, -4.03527030e+28, -2.29825444e+28,  4.89421934e+28,
       -2.74989793e+27,  1.55802029e+27,  3.18580134e+28,  1.84321074e+28,
        2.03934250e+28,  3.32140473e+26])
 message: 'Optimization terminated successfully.'
     nit: 2
 success: True
       x: array([-150.36690891,  -17.97830665,   45.19175556, -127.89554336,
         43.12614282,   -6.85613009, -121.83253146,   18.70751814,
         43.21144455,   42.47319733,  -53.2714565 ,  -62.70138486,
        -67.27681025,   63.10708298,  188.62118607, -119.17756975,
       -199.8673778 ,  -86.06314987])

In [15]:
if hasattr(result,'x'): 
    x = result.x
else:
    x = result.population[0]
    x = (x-0.5)*scale + mid

In [16]:
print(objFunc([0]*nSext))
M0,G0 = mli.readTransferMap()

1.5219419845303284e+34


In [17]:
print(objFunc(SextStrength))
M1,G1 = mli.readTransferMap()

9.689780484408821e+31


In [18]:
print(objFunc(x))
M2,G2 = mli.readTransferMap()

2.0107957494427288e+31


In [19]:
G0.head()

,exponents,GP
28,f( 30 00 00 ),0.964896
29,f( 21 00 00 ),1.776233
33,f( 20 00 01 ),-50.498113
34,f( 12 00 00 ),0.391981
38,f( 11 00 01 ),-75.263169


In [20]:
G1.head()

,exponents,GP
28,f( 30 00 00 ),7.765582e-12
29,f( 21 00 00 ),-5.701697e-12
33,f( 20 00 01 ),7.272738e-01
34,f( 12 00 00 ),-2.987872e-12
38,f( 11 00 01 ),5.580342e-12


In [21]:
G2

,exponents,GP
28,f( 30 00 00 ),1.610494e+00
29,f( 21 00 00 ),-1.549822e+00
33,f( 20 00 01 ),-3.974357e-01
34,f( 12 00 00 ),8.202024e-01
38,f( 11 00 01 ),6.803332e-01
39,f( 10 20 00 ),3.941719e-02
40,f( 10 11 00 ),-3.888254e-01
43,f( 10 02 00 ),5.925154e-01
48,f( 10 00 02 ),3.647279e-01
49,f( 03 00 00 ),-1.355843e-01
